# Very Random Coding Practices

## Design Questions

### Efficiently store and calculate dot products between embeddings
<p></p>
- [leetcode source](https://leetcode.com/discuss/interview-question/286446/facebook-interview-question-representation-and-dot-product-of-two-vectors)

    A: [1, 1, 4, 4, 4, 4, 7, 7, 7, 7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

    B: [2, 2, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [ ]:
def compress_vector(A):
    if len(A) == 0:
        return []
    
    compress_A = [[A[0], 1]] # [value, count]
    for a in A[1:]:
        if a == compress_A[-1][0]:
            compress_A[-1][1] += 1
        else:
            compress_A.append([a, 1])
    return compress_A

def calculate_dot_product(A, B):
    
    compress_A = compress_vector(A)
    compress_B = compress_vector(B)
    
    i, j, dot_prod = 0, 0, 0
    len_A, len_B = len(compress_A), len(compress_B)
    while i < len_A and j < len_B:
        a, b = compress_A[i], compress_B[j]
        common = min(a[1], b[1])
        dot_prod += a[0] * b[0] * common
        
        a[1] -= common
        b[1] -= common
        
        if a[1] == 0:
            i += 1
        if b[1] == 0:
            j += 1
    return dot_prod


A = [1, 1, 4, 4, 4, 4, 7, 7, 7, 7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
B = [2, 2, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

assert calculate_dot_product(A, B) == sum([a * b for a, b in zip(A, B)])

### Design a sparse matrix class

[1, 0, 0, 0, 0]

[0, 2, 0, 0, 0]

[0, 0, 0, 0, 1]

[0, -1, 0, 0, 0]

In [23]:
class SparseMatrix:
    
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.matrix = defaultdict(float)
    
    def get(self, x, y):
        if 0<=x<self.rows and 0<=y<self.cols:
            if (x, y) in self.matrix:
                return self.matrix[(x, y)]
            else:
                return 0
        else:
            raise Exception("invalid input range")
    
    def set(self, x, y, val):
        if 0<=x<self.rows and 0<=y<self.cols:
            if val == 0:
                if (x, y) in self.matrix:
                    del self.matrix[(x, y)]
            else:
                self.matrix[(x, y)] = val
        else:
            raise Exception("invalid input range")
    
    
    def add(self, B):
        if self.rows != B.rows or self.cols != B.cols:
            raise Exception("Incompatible matrix size")
        for i in range(self.rows):
            for j in range(self.cols):
                temp = self.get(i, j) + B.get(i, j)
                self.set(i, j, temp)
    
    def multiply(self, B):
        pass
    

### Build K-means from scratch using numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class kMeansNumpy:
    def __init__(self, k, dim=2):
        self.k = k
        self.centriod = np.zeros((dim, k))
    
    def fit(self, X, iteration=2):
        self.X = X
        self.cluster = np.zeros(len(self.X))
        self._initialize()
        fig, axs = plt.subplots(iteration+1,figsize=(5,15))
        self._plot(axs[0])
        
        plt_counter = 0
        while iteration >= 0:
            self._update_centriod()
            self._assign_cluster()
            self._plot(axs[plt_counter])
            plt_counter += 1
            iteration -= 1
        
    def _assign_cluster(self):
        for i, x in enumerate(self.X):
            distance = []
            for c in self.centriod:
                distance.append(self._get_distance(x, c))
            self.cluster[i] = np.argmin(distance)
        print('updated cluster is {}'.format(self.cluster))
        
    def _update_centriod(self):
        for c in range(self.k):
            X = self.X[self.cluster == c,]
            self.centriod[c] = np.mean(X,axis=0)
        print('updated cnetrioids are {}'.format(self.centriod))
        
    def _get_distance(self, x, y):
        return np.linalg.norm(x-y)
    
    def _initialize(self):
        m, _ = self.X.shape
        initial_k = np.random.choice(m, self.k, replace=False)
        self.centriod = self.X[initial_k,:]
        print('initialized cnetrioids are {}'.format(self.centriod))
        self._assign_cluster()
        
    def _plot(self, axis):
        axis.scatter(x=self.X[:, 0], y=self.X[:, 1], c=self.cluster)
        
X = np.array([[2, 3], [4, 5], [7, 7], [2, 7], [10, 2], [12, 2], [3, 2], [-3, 9], [0, 8], [1, 7], [4, 9], [-4, -3]])

classifier = kMeansNumpy(3)
classifier.fit(X)

### Build K-means from scratch using PySpark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

In [ ]:
class kMeansSpark:
    def __init__(self, k):
        self.k = k
    
    def _update_cluster(self):
        pass
    
    def _assign_centriod(self):
        pass
    
    def fit(self):
        pass

### Build K-NN from scratch

In [2]:
import math
import heapq

def get_distance(x, y):
    distance = 0.0
    for i in range(len(x)-1):
        distance += (x[i] - y[i]) ** 2
    return math.sqrt(distance)

def get_neighbors(train, test, num_neigbhors):
    distances = list()
    for train_data in train:
        dist = get_distance(train, test)
        heapq.heappush(distances, (dist, train))
        if len(distances) > num_neigbhors:
            heapq.heappop(distances)
    res = []
    while distances:
        _, train = heapq.heappop(distances)
        res.append(train)
    return res

def make_prediction(train, test, num_neigbhors):
    neigbhors = get_neighbors(train, test, num_neigbhors)
    classes = [row[-1] for row in neigbhors]
    prediction = max(set(classes), key=classes.count)
    return prediction

### Find k-closest points to origin
- [leetcode source](https://leetcode.com/problems/k-closest-points-to-origin/)
<p></p>
- three solutions depending on problem scale: is $N$ very big? is $K$ big or small?
<p></p>

    a) use a priority queue to maintain a list of k points, $O(Nlog(k))$;
    
    b) apply sorting, $O(Nlog(N))$;
    
    c) apply kd tree, $O(Nlog(N))$ for the first time to generate the tree, search, insert, remove all takes $O(log(N))$.
<p></p>
- a good practice on heapq

In [ ]:
import heapq

def find_k_closest_heap(points, k):
    candidates = []
    for x, y in points:
        dist = - x*x - y*y
        if len(candidates) == k:
            heapq.heappushpop(candidates, (dist, x, y))
        else:
            heapq.heappush(candidates, (dist, x, y))
    
    return [[x, y] for dist, x, y in candidates]


from scipy import spatial

def find_k_closest_kd_tree(points, k):
    kd_tree = spatial.KDTree(points)
    distance, idx = kd_tree.query(x=[0, 0], k=k, p=2)
    if k > 1:
        return [point for i, point in enumerate(points) if i in idx]
    else:
        return [points[idx]]
print(find_k_closest_heap([[3,3],[5,-1],[-2,4]], 2))
print(find_k_closest_kd_tree([[3,3],[5,-1],[-2,4]], 2))

## Maths Questions

### Inverse a matrix from scratch
1. loop over a focus_diagonal, e.g., 1,2,3;
2. for each focus_diagonal, loop over the other rows, deduct the current_row scalar from it;
3. list.copy() only copies this outter list, if it is a list of list, it doesn't copy the inner lists. copy.deepcopy() should be used instead.

In [ ]:
import numpy as np
import copy

def inverse_matrix(A):
    
    if not check_matrix_squareness(A):
        return []
    
    n = len(A)
    AM = copy.deepcopy(A)
    I = get_identity_matrix(n)
    row_index = list(range(n))
        
    for fd in range(n):
        try: 
            fd_scalar = 1.0 / AM[fd][fd]
        except ZeroDivisionError:
            return []
            
        for j in range(n):
            AM[fd][j] *= fd_scalar
            I[fd][j] *= fd_scalar
        
        for i in row_index[:fd]+row_index[fd+1:]:
            cr_scalar = AM[i][fd]
            for j in range(n):
                AM[i][j] = AM[i][j] - cr_scalar * AM[fd][j]
                I[i][j] = I[i][j] - cr_scalar * I[fd][j]

    return I

def get_identity_matrix(n):
    I = []
    for i in range(n):
        row = []
        for j in range(n):
            if i == j:
                row.append(1)
            else:
                row.append(0)
        I.append(row)
    return I


def check_matrix_squareness(A):
    if len(A) == 0 or len(A[0]) == 0 or len(A) != len(A[0]):
        return False
    else:
        return True
    
A = [[1,5,3], [4,5,6], [7,8,9]]

print(inverse_matrix(A))
print(np.linalg.inv(A))

### Check if a matrix is full rank

In [ ]:
A = [[1,2,3], [2,4,6], [7,8,9]]
if not inverse_matrix(A):
    print("A is not full rank")

### Solve a polynomial equation

In [ ]:
# use recursion

### Find square root of N with precision P

In [9]:
import math

def get_square_root(N, P):
    guess = N / 2
    while abs(guess * guess - N) > P:
        guess = (guess + N/guess) / 2
    return guess

print(get_square_root(5, 0.0001), math.sqrt(5))

2.2360679779158037 2.23606797749979


## Statistics Questions

### Write Batch Normalization from scratch

In [ ]:
import math

def batch_norm_dense(layer: list, epsilon: float, ita: float, beta: float):
    total = sum(layer)
    mu = total / len(layer)
    sigma = 0
    for ele in layer:
        sigma += (ele-mu)**2
    sigma /= len(layer)
    
    output = []
    for ele in layer:
        norm_ele = (ele - mu) / math.sqrt(sigma**2 + epsilon)
        output.append(ita * norm_ele + beta)
    return output

batch_norm_dense([1,2,3,4,5], 0.0001, 0.05, 0.3)

### Calculate precision, recall, AUC

In [ ]:
import matplotlib.pyplot as plt

def get_metrics(pred, true, threshold=0.5):
    pred_class = []
    for ele in pred:
        if ele >= threshold:
            pred_class.append(1)
        else:
            pred_class.append(0)
    
    TP, FN = 0, 0
    FP, TN = 0, 0
    for p, t in zip(pred_class, true):
        if t == 1:
            if p == 1:
                TP += 1
            else:
                FN += 1
        else:
            if p == 1:
                FP += 1
            else:
                TN += 1
    precision = TP / (TP + FP) if (TP+FP) else 1
    recall = TP / (TP + FN) if (TP+FN) else 1
    
    return precision, recall

def get_auc(pred, true):
    auc_pair = []
    plt.figure()
    for i in range(1, 11):
        auc_pair.append(get_metrics(pred, true, i/10))
    
    auc_pair.sort(key=lambda x: x[0])
    plt.step(auc_pair[0], auc_pair[1])

get_auc([0.7, 0.6, 0.3, 0.6, 0.1, 0.9], [1, 0, 0, 0, 1, 1])

### Get mean and variance from a large data stream

In [2]:
# find the relationship between (mean_k+1 and mean_k), (var_k+1 and var_k)

### Write CART

In [31]:
# Importing the required packages
import numpy as np
import pandas as pd
import itertools

# Reading and cleaning the data
# titanic = pd.read_csv("titanic.csv")
# target = titanic["Survived"]
# features = titanic.loc[:, ["Pclass", "Sex", "Embarked"]]
# features.Embarked.fillna(features.Embarked.mode()[0], inplace=True)
# features.Pclass = features.Pclass.map({1: "1st", 2: "2nd", 3: "3rd"})

In [33]:
# Main class
class Node:
    def __init__(self, features, target):
        self.left = None 
        self.right = None
        self.features = features
        self.target = target 
        self.feature_types = self.select_dtype()
        
    # Identiy feature types
    def select_dtype(self):
        feature_types = []
        for item in self.features.columns:
            if self.features[item].dtype != "O":
                feature_types.append((item, "numerical")) 
            else:
                if self.features[item].nunique() <= 2:
                    feature_types.append((item, "binary"))
                else:
                    feature_types.append((item, "multiclass"))
        return feature_types
    
    # Calculate gini impurity for each feature at a node
    # Need to provide a default value as a fallback 
    @staticmethod
    def gini_impurity_total(a=0, b=0, c=0, d=0):
        total_elements = a + b + c + d
        gini_1 = 1 - np.square(a/(a+b)) - np.square(b/(a+b))
        gini_2 = 1 - np.square(c/(c+d)) - np.square(d/(c+d))
        total_gini = ((a+b)/total_elements) * gini_1 + ((c+d)/total_elements) * gini_2
        return total_gini 
    
    @staticmethod
    def gini_impurity(a=0, b=0):
        return 1 - np.square(a/(a+b)) - np.square(b/(a+b))
    
    # Calculate gini for all feature combinations in categorical features
    def calculate_gini(self, feature):
        gini_node = []
        combinations = []
        
        for i in range(1, self.features[feature].nunique()):
            combinations = combinations + list(itertools.combinations(self.features[feature].unique(), i))
            
        for item in combinations:
            t1 = self.target[self.features[feature].isin(item)] 
            t2 = self.target[~self.features[feature].isin(item)]
            args = t1.value_counts().tolist() + t2.value_counts().tolist()
            gini_node.append(Node.gini_impurity_total(*args))
        
        return gini_node, combinations # Return all the values 
        
# Get the best gini values for each feature  
    def evaluate_node(self):
        gini_values = []
        for feature in self.features.columns:
            calculated_gini, combinations = self.calculate_gini(feature)
            best_combination = combinations[np.argmin(calculated_gini)]
            gini_values.append((feature, best_combination, np.min(calculated_gini)))
        return gini_values
                
    # Inserting a new node based on the decision criteria
    def insert_node(self):
        gini_values = self.evaluate_node()
        values = [item[2] for item in gini_values]
        node_gini =  Node.gini_impurity(*self.target.value_counts().tolist())
        
         # Terminate the branch if current gini is better or no features to split
        if node_gini < np.min(values): 
            print("terminating the branch")
            self.left = None
            self.right = None 
        else:
            best_feature = gini_values[np.argmin(values)][0]
            best_combination = gini_values[np.argmin(values)][1]
            print(f"Creating a new branch using {best_feature} and {best_combination}")

            left_features = self.features[self.features[best_feature].isin(best_combination)]
            left_features.drop([best_feature], axis=1, inplace=True)
            left_target = self.target[self.features[best_feature].isin(best_combination)]
            if list(left_features.columns) == []:
                self.left = None 
                self.right = None
            else:
                self.left = Node(left_features, left_target)
                self.left.insert_node()

            right_features = self.features[~self.features[best_feature].isin(best_combination)]
            right_target = self.target[~self.features[best_feature].isin(best_combination)]
            right_features.drop([best_feature], axis=1, inplace=True)
            if list(right_features.columns) == []:
                self.left = None 
                self.right = None
            else:
                self.right = Node(right_features, right_target)
                self.right.insert_node()

# Creating the root node with the full dataset 
# root = Node(features, target)

# Building the tree 
# root.insert_node()

## Algo Basics

### Sorting

In [ ]:
class Sort:
    def __init__(self, nums):
        self.nums = nums
    
    def do_quick_sort(self):
        self.quick_sort(0, len(self.nums)-1)
        return self.nums

    
    def partition(self, l, h):
        pivot = self.nums[h]
        pivot_i = l-1

        for i in range(l, h):
            if self.nums[i] < pivot:
                pivot_i += 1
                self.nums[pivot_i], self.nums[i] = self.nums[i], self.nums[pivot_i]
                

        self.nums[pivot_i+1], self.nums[h] = self.nums[h], self.nums[pivot_i+1]
        return pivot_i+1


    def quick_sort(self, l, h):
        if l < h:
            pivot_i = self.partition(l, h)
            self.quick_sort(l, pivot_i-1)
            self.quick_sort(pivot_i+1, h)



Sort([4,9,1,6,7,982,6,3,1]).do_quick_sort()

## Non-Leetcode Coding Questions

### Enumerate/Guess password combinations
Given the length of password n, and a list of unique digits where each must appear at least once. Guess the password

In [28]:
import random

def check(x):
    return x == "12312"

def guess_password_monte_carlo(n=5, digits=[1,2,3]):
    counter = 0
    cache = set()
    while True:
        counter += 1
        guess = ""
        for i in range(n):
            guess += str(random.choice(digits))
        if len(set(guess)) == len(digits) and guess not in cache:
            if check(guess):
                print(guess)
                return
            else:
                cache.add(guess)

def guess_password_backtrack(n=5, digits=[1,2,3]):
    
    def backtrack(path, unique_path):

        # early stopping
        if n - len(path) < len(digits) - len(unique_path):
            return
        
        # termination
        if len(path) == n:
            if len(unique_path) == len(digits):
                guess = "".join(path)
                if check(guess):
                    print(guess)
            return
        
        # enumerate
        for j in range(len(digits)):
            path.append(str(digits[j]))
            if str(digits[j]) in unique_path:
                unique_path[str(digits[j])] += 1
            else:
                unique_path[str(digits[j])] = 1
                
            # backtrack
            backtrack(path, unique_path)
            
            # revert
            path.pop()
            unique_path[str(digits[j])] -= 1
            if unique_path[str(digits[j])] == 0:
                del unique_path[str(digits[j])]

    backtrack([], {})
    

    
#         def backtrack(first = 1, curr = []):
#             # if the combination is done
#             if len(curr) == k:  
#                 output.append(curr[:])
#             for i in range(first, n + 1):
#                 # add i into the current combination
#                 curr.append(i)
#                 # use next integers to complete the combination
#                 backtrack(i + 1, curr)
#                 # backtrack
#                 curr.pop()

guess_password_monte_carlo()
guess_password_backtrack()


12312
12312


### Maze with Monsters 
A charcter walks from si, sj to ei, ej, return the shortest steps if possible.
A 2D array with:

0: empty space to walk through;

1: a wall that can not be passed;

6: a monster that reduces your health if you pass.

In [24]:
from collections import deque

def get_shortest_steps(maze, h, si, sj, ei, ej):
    m, n = len(maze), len(maze[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    health_status = [[float("-Inf")]*n for _ in range(m)]
    
    queue = deque([(0, h, si, sj)])
    seen = set((0, 0, h))
    
    while queue:
        cur_step, cur_h, i, j = queue.popleft()
        
        # termination
        if i == ei and j == ej:
            return cur_step, cur_h
        
        for direction in directions:
            x, y = i + direction[0], j + direction[1]
            if 0<=x<m and 0<=y<n and maze[x][y] != 1:
                if maze[x][y] == 0:
                    cur_status = (x, y, cur_h)
                    if cur_status not in seen:
                        seen.add(cur_status)
                        queue.append((cur_step+1, cur_h, x, y))
                elif maze[x][y] == 6:
                    new_h = cur_h - 1
                    cur_status = (x, y, new_h)
                    if new_h > 0 and cur_status not in seen:
                        seen.add(cur_status)
                        queue.append((cur_step+1, new_h, x, y))
    return -1, -1
                
                
maze = [[0,0,0],[1,1,6],[0,0,0],[6,1,1],[0,0,0]]
print(get_shortest_steps(maze, 3, 0, 0, 4, 2))

(10, 1)


### Maximum Path Sum from Leaf Nodes
follow-up: what if it is between any two asterisk node

In [7]:
class TreeNode:
    def __init__(self, val=0):
        self.val = val
        self.left = None
        self.right = None

n_n10 = TreeNode(-10)
n_n3 = TreeNode(-3)
n_20 = TreeNode(20)
n_15 = TreeNode(15)
n_n7 = TreeNode(-7)

n_n10.left = n_n3
n_n10.right = n_20
n_20.left = n_15
n_20.right = n_n7

def getMaxSumfromLeaf(root):
    if not root:
        return 0
    
    res = float("-Inf")
    
    def recursion(node):
        
        nonlocal res
        
        if not node:
            return 0
        
        # becasue it is leaf-leaf, we have to preserve all sum from child even they are negative
        left_sum = recursion(node.left)
        right_sum = recursion(node.right)
        
        # only update when current node is in the middle
        if node.left and node.right:
            res = max(res, left_sum+right_sum+node.val)
        
        return max(left_sum, right_sum) + node.val
    
    recursion(root)
    return res

print(getMaxSumfromLeaf(n_n10))

28


### Find Tree Difference

In [14]:
class TreeNode:
    def __init__(self, val=0, key=None, active=False):
        self.val = val
        self.key = key
        self.active = active
        self.children = []

A1 = TreeNode(val=1, key="a", active=True)
A2 = TreeNode(val=2, key="b", active=True)
A3 = TreeNode(val=3, key="c", active=True)
A4 = TreeNode(val=4, key="d", active=True)
A5 = TreeNode(val=5, key="e", active=True)
A6 = TreeNode(val=6, key="f", active=True)

A1.children = [A2, A3]
A2.children = [A4, A5]
A3.children = [A6]

B1 = TreeNode(val=1, key="a", active=True)
B3 = TreeNode(val=3, key="c", active=False)
B66 = TreeNode(val=66, key="f", active=True)

B1.children = [B3]
B3.children = [B66]

        
def get_tree_diff(A, B):
    
    
    def get_tree_map(node, tree_map):
        
        if not node:
            return
        tree_map[node.key] = (node.active, node.val)
        for child in node.children:
            get_tree_map(child, tree_map)
    
    def get_diff(node, tree_map):
        if not node:
            return 0
            
        count = 0
        if node.key not in tree_map:
            count += 1
        else:
            if tree_map[node.key][1] != node.val:
                count += 1
            else:
                if (not tree_map[node.key][0] and node.active) or (tree_map[node.key][0] and not node.active):
                    count += 1
            del tree_map[node.key]
            
        for child in node.children:
            count += get_diff(child, tree_map)
        return count
    
    a_map = {}
    
    get_tree_map(A, a_map)
    
    return get_diff(B, a_map) + len(a_map)
        
print(get_tree_diff(A1, B1))

5


### Alien Dictionary Print All Solutions

In [8]:
from collections import defaultdict

def alien_dict(words):
    
    # build graph with all unique char and degrees
    degrees = {}
    for word in words:
        for char in word:
            degrees[char] = 0
    unique_count = len(degrees)
    
    graph = defaultdict(set)
    for word0, word1 in zip(words[:-1], words[1:]):
        l = len(word1)
        if len(word0) > len(word1) and word0[:l] == word1:
            return []
        
        for i in range(len(word0)):
            if word0[i] != word1[i]:
                if word1[i] not in graph[word0[i]]:
                    graph[word0[i]].add(word1[i])
                    degrees[word1[i]] += 1
                break
    
    def dfs(path):
        
        # termination
        if len(path) == unique_count:
            res.append("".join(path))
            return
        
        # enumerate to add all others from the same degrees
        for node in degrees:
            if node not in path and degrees[node] == 0:
                path.append(node)
                dfs(path)
                path.pop()
            
        # enumerate to next degree level
        for adj in graph[path[-1]]:
            degrees[adj] -= 1
            if degrees[adj] == 0:
                path.append(adj)
                del degrees[adj]
                dfs(path)
                degrees[adj] = 0
                path.pop()

    res = []
    for char in degrees:
        if degrees[char] == 0:
            dfs([char])
    
    return res

words = ["ab","bc"]
print(alien_dict(words))

['abc', 'cab']


### Trie to store bad phases

In [17]:
from collections import defaultdict

class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.is_end = False

class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, sentence):
        node = self.root
        words = sentence.split(" ")
        for word in words:
            node = node.children[word]
        node.is_end = True
    
    def search(self, sentence):
        words = sentence.split(" ")
        return self._search(words)
    
    def _search(self, words):
        node = self.root
        for i, word in enumerate(words):
            if word in node.children:
                node = node.children[word]
                if node.is_end:
                    return True
            else:
                return self._search(words[i+1:])
        return False

bad_phrases = ["I like violence", "I like violence too", "gore", "world war i", "world war ii"]
queries = ["foo I like violence bar", "like violence i", "world war is bad"]

detector = Trie()
for bad_phrase in bad_phrases:
    detector.insert(bad_phrase)

res = []
for query in queries:
    res.append(detector.search(query))
print(res)

[True, False, False]
